In [1]:
import tensorflow as tf
import numpy as np
import sys

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
n_input = 28 * 28
batch_size = 100
training_epochs = 2
display_step = 1

log_dir = '/tmp/tensorflow/Tensorboard/logs'

In [32]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
reset_graph()    

In [33]:

X = tf.placeholder(tf.float32, [None, n_input])


l2_reg = 0.005
he_init = tf.contrib.layers.variance_scaling_initializer() # He initialization
l2_regularizer = tf.contrib.layers.l2_regularizer(l2_reg) 

with tf.name_scope('layer_enc'):
    layer_enc = tf.layers.dense(X, 50, name='layer1')

with tf.name_scope('layer_dec'):
    layer_dec = tf.layers.dense(layer_enc, n_input, name='layer2')

with tf.name_scope('prediction'):
    y_pred = tf.nn.sigmoid(layer_dec)

y_true = X

with tf.name_scope('sigmoid_cross_entropy'):
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=layer_dec)

    cost = tf.reduce_mean(loss)
learning_rate = 0.001
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [76]:
name = layer_dec.graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
#'layer_dec/layer2/BiasAdd:0'

test = layer_dec.op.graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

In [79]:
test[0]

<tf.Variable 'layer1/kernel:0' shape=(784, 50) dtype=float32_ref>

In [35]:
# Initializing the variables
#merged = tf.summary.merge_all()
init = tf.global_variables_initializer()

if tf.gfile.Exists(log_dir):
    tf.gfile.DeleteRecursively(log_dir)
tf.gfile.MakeDirs(log_dir)

saver = tf.train.Saver()
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    train_writer = tf.summary.FileWriter(log_dir + '/summaries/train', sess.graph)
    test_writer = tf.summary.FileWriter(log_dir + '/summaries/test')

    total_batch = int(mnist.train.num_examples/batch_size)
    # Training cycle
    for epoch in range(training_epochs):
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) adn cost op (to get loss value)
            #summary, _, c = sess.run([merged, optimizer, cost], feed_dict={X: batch_xs})
            _, c = sess.run([optimizer, cost], feed_dict={X: batch_xs})
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c))
            sys.stdout.flush()
        #train_writer.add_summary(summary, epoch)
    print("Optimization Finished!")
    train_writer.close()
    test_writer.close()
    saver.save(sess, log_dir + "model/my_model")

Epoch: 0001 cost= 0.140479743
Epoch: 0002 cost= 0.103864700
Optimization Finished!


In [36]:
for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(v.name)
    
"""
layer_enc/layer1/kernel:0
layer_enc/layer1/bias:0
layer_dec/layer2/kernel:0
layer_dec/layer2/bias:0
"""

layer1/kernel:0
layer1/bias:0
layer2/kernel:0
layer2/bias:0


'\nlayer_enc/layer1/kernel:0\nlayer_enc/layer1/bias:0\nlayer_dec/layer2/kernel:0\nlayer_dec/layer2/bias:0\n'

In [37]:
layer_dec.name

'layer_dec/layer2/BiasAdd:0'